In [12]:
import logging
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight
from keras.callbacks import *

In [13]:
import gensim

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

In [3]:
MODEL_PATH = "../models/"

In [4]:
DATA_PATH = "../data/"
X_train = pickle.load(open(DATA_PATH + "X_train.p", "rb"))
X_dev = pickle.load(open(DATA_PATH + "X_dev.p", "rb"))
y_train = pickle.load(open(DATA_PATH + "y_train.p", "rb"))
y_dev = pickle.load(open(DATA_PATH + "y_dev.p", "rb"))

In [5]:
y_train.shape

(111699, 6)

In [6]:
train_text = X_train['comment_text']

In [7]:
tokenizer = Tokenizer(num_words=18400)
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)

In [15]:
word_index = tokenizer.word_index
from gensim.models.wrappers import FastText
embeddings_index = FastText.load_word2vec_format('../../../embeddings/crawl-300d-2M.vec')

In [24]:
print('Found %s word vectors.' % len(embeddings_index.vocab))
embedding_matrix = np.zeros((len(word_index) + 1, 300))
oov = []
for word, i in word_index.items():
    try:
        embedding_vector = embeddings_index[word]
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    except Exception as e:
        oov.append(word)
embedding_size = 300
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=200, trainable = False, mask_zero = True)

Found 2000000 word vectors.


In [19]:
len(oov)

80572

In [21]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_dev_texts = tokenizer.texts_to_sequences(X_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

In [28]:
batch_norm = False
num_ensembles = 11

def create_lstm_model():
    sequence_input = Input(shape=(max_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    lstm1 = LSTM(20, return_sequences=True)(embedded_sequences)
    lstm2 = LSTM(5)(lstm1)
    output = Dense(units=6, activation='sigmoid')(lstm2)
    return Model(sequence_input, output)

lstm_models = [create_lstm_model() for i in range(0,num_ensembles)]

ensemble_input = Input(shape=(max_length,))
concat = keras.layers.Concatenate()([lstm_model(ensemble_input) for lstm_model in lstm_models])
# averaged = keras.layers.average([lstm_model(ensemble_input) for lstm_model in lstm_models])
weighted = Dense(units=1)(concat)
ensemble_lstm = Model(inputs=[ensemble_input], outputs=[weighted])
ensemble_lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    


ensemble_lstm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
model_12 (Model)                (None, 6)            50675536    input_28[0][0]                   
__________________________________________________________________________________________________
model_13 (Model)                (None, 6)            50675536    input_28[0][0]                   
__________________________________________________________________________________________________
model_14 (Model)                (None, 6)            50675536    input_28[0][0]                   
__________________________________________________________________________________________________
model_15 (

In [29]:
model_name = "keras_lstm-dense_fasttext_ensemble_classification_model"

In [ ]:
# model_name = "cnn_hannes"
# check_point_path= "../models/" + model_name + ".h5"
# callbacks = [
#      ModelCheckpoint(check_point_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# ]
for model in lstm_models:
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
    ]
    model.fit(x_train_texts, y_train, validation_data=(x_dev_texts, y_dev),
              epochs=15, batch_size=50, callbacks = callbacks)

Train on 111699 samples, validate on 47872 samples
Epoch 1/15
111699/111699 [==============================] - 2141s 19ms/step - loss: 0.1127 - acc: 0.9716 - val_loss: 0.0598 - val_acc: 0.9801
Epoch 2/15
111699/111699 [==============================] - 2206s 20ms/step - loss: 0.0553 - acc: 0.9806 - val_loss: 0.0534 - val_acc: 0.9809
Epoch 3/15
111699/111699 [==============================] - 3148s 28ms/step - loss: 0.0511 - acc: 0.9816 - val_loss: 0.0509 - val_acc: 0.9817
Epoch 4/15
111699/111699 [==============================] - 3060s 27ms/step - loss: 0.0486 - acc: 0.9822 - val_loss: 0.0492 - val_acc: 0.9820
Epoch 5/15
111699/111699 [==============================] - 3057s 27ms/step - loss: 0.0472 - acc: 0.9825 - val_loss: 0.0495 - val_acc: 0.9821
Epoch 6/15
111699/111699 [==============================] - 3087s 28ms/step - loss: 0.0456 - acc: 0.9829 - val_loss: 0.0484 - val_acc: 0.9823
Epoch 7/15
111699/111699 [==============================] - 3059s 27ms/step - loss: 0.0443 - acc:

In [32]:
model_name = 'keras_ensemble_lstm_classification_model'

In [33]:
model_json = model.to_json()
with open(MODEL_PATH + model_name + ".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
ensemble_lstm.save_weights(MODEL_PATH + model_name + ".h5")
print("Saved model to disk")
# ensemble_cnn.load_weights(check_point_path)

Saved model to disk


In [35]:
pred_dev = ensemble_lstm.predict(x_dev_texts)

In [36]:
def mean_roc_auc(y_true, y_pred):
    roc_auc_scores = []
    for i in range(0, y_true.shape[1]):
        roc_auc_scores.append(metrics.roc_auc_score(y_true[:, i], y_pred[:, i]))
    print(roc_auc_scores)
    return np.mean(roc_auc_scores)

In [38]:
y_dev

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [37]:
mean_roc_auc(y_dev, pred_dev)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
import pandas as pd

In [ ]:
X_official_test = pd.read_csv("../data/raw/test.csv")
x_dev_texts = tokenizer.texts_to_sequences(X_official_test['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')
pred = model.predict(x_dev_texts)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission = pd.read_csv('../submissions/sample_submission.csv')
sample_submission[list_classes] = pred
sample_submission.to_csv("../submissions/" + model_name + ".csv", index=False)